In [1]:
from pathlib import Path

from gbd_mapping import causes, risk_factors, sequelae
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from vivarium import Artifact, InteractiveContext

from vivarium_nih_us_cvd.constants import data_keys
from vivarium_nih_us_cvd.data import loader

In [2]:
path_repo = Path('/ihme/homes/sbachmei/repos/vivarium_nih_us_cvd/src/vivarium_nih_us_cvd/')
# path_artifact = Path('/ihme/homes/sbachmei/scratch/vivarium/alabama.hdf')
path_artifact = Path('/ihme/costeffectiveness/artifacts/vivarium_nih_us_cvd/alabama.hdf')
path_config = path_repo / 'model_specifications/nih_us_cvd.yaml'

# Simulation

In [3]:
sim = InteractiveContext(path_config)

2022-09-23 09:04:49.789 | DEBUG    | vivarium.framework.values:register_value_modifier:392 - Registering metrics.1.population_manager.metrics as modifier to metrics
2022-09-23 09:04:49.900 | DEBUG    | vivarium.framework.artifact.manager:_load_artifact:66 - Running simulation from artifact located at /ihme/costeffectiveness/artifacts/vivarium_nih_us_cvd/alabama.hdf.
2022-09-23 09:04:49.902 | DEBUG    | vivarium.framework.artifact.manager:_load_artifact:67 - Artifact base filter terms are ['draw == 0'].
2022-09-23 09:04:49.911 | DEBUG    | vivarium.framework.artifact.manager:_load_artifact:68 - Artifact additional filter terms are None.
2022-09-23 09:04:50.612 | DEBUG    | vivarium.framework.values:_register_value_producer:338 - Registering value pipeline cause_specific_mortality_rate
2022-09-23 09:04:50.613 | DEBUG    | vivarium.framework.values:_register_value_producer:338 - Registering value pipeline mortality_rate
2022-09-23 09:04:51.079 | DEBUG    | vivarium.framework.values:_regis

## Initialization

In [4]:
pop = []
metrics = []

pop.append(sim.get_population())
idx = pop[0].index
metrics.append(sim.get_value('metrics')(idx))

In [5]:
# Investigate state table for new columns
pop0 = pop[0].copy()

In [6]:
pop0.columns

Index(['tracked', 'age', 'exit_time', 'alive', 'location', 'sex',
       'entrance_time', 'years_of_life_lost', 'cause_of_death',
       'previous_ischemic_stroke', 'previous_myocardial_infarction',
       'ischemic_stroke', 'myocardial_infarction',
       'high_ldl_cholesterol_propensity',
       'ensemble_propensity_risk_factor.high_ldl_cholesterol',
       'high_systolic_blood_pressure_propensity',
       'ensemble_propensity_risk_factor.high_systolic_blood_pressure',
       'susceptible_to_ischemic_stroke_event_count',
       'susceptible_to_ischemic_stroke_event_time',
       'acute_ischemic_stroke_event_count', 'acute_ischemic_stroke_event_time',
       'chronic_ischemic_stroke_event_time',
       'chronic_ischemic_stroke_event_count',
       'susceptible_to_myocardial_infarction_event_time',
       'susceptible_to_myocardial_infarction_event_count',
       'acute_myocardial_infarction_event_time',
       'acute_myocardial_infarction_event_count',
       'post_myocardial_infarcti

In [7]:
cols = [
    'visit_type',
    'scheduled_date',
    'miss_scheduled_visit_probability',
    'sbp_medication',
    'ldlc_medication',
    'sbp_medication_adherence',
    'ldlc_medication_adherence',
]

In [8]:
pop0[cols]

,visit_type,scheduled_date,miss_scheduled_visit_probability,sbp_medication,ldlc_medication,sbp_medication_adherence,ldlc_medication_adherence
0,none,NaT,0.161931,1.0,2.0,adherent,adherent
1,none,NaT,0.262446,NaN,NaN,adherent,primary_non_adherent
2,none,NaT,0.278118,NaN,NaN,primary_non_adherent,primary_non_adherent
3,none,NaT,0.263973,NaN,NaN,adherent,primary_non_adherent
4,none,NaT,0.120069,NaN,NaN,adherent,adherent
...,...,...,...,...,...,...,...
9995,none,NaT,0.221730,NaN,NaN,adherent,adherent
9996,none,NaT,0.209327,1.0,2.0,adherent,adherent
9997,none,NaT,0.255036,NaN,NaN,adherent,primary_non_adherent
9998,none,2021-02-04 03:46:42.277641600,0.163858,1.0,2.0,adherent,adherent


## medication baseline coverage

In [9]:
# get pipeline values
sbp = sim.get_value('high_systolic_blood_pressure.exposure')(pop0.index)
ldlc = sim.get_value('high_ldl_cholesterol.exposure')(pop0.index)

In [10]:
# baseline treatment coverage covariates
c_sbp = np.exp(-6.75 + 0.025 * sbp - 0.0045 * ldlc + 0.05 * pop0.age + 0.16 * pop0.sex.map({"Female": 2, "Male": 1}))
c_ldlc = np.exp(-4.23 - 0.0026 * sbp - 0.005 * ldlc + 0.062 * pop0.age - 0.19 * pop0.sex.map({"Female": 2, "Male": 1}))
c_both = np.exp(-6.26 + 0.018 * sbp - 0.014 * ldlc + 0.069 * pop0.age + 0.13 * pop0.sex.map({"Female": 2, "Male": 1}))

In [11]:
denom = c_sbp + c_ldlc + c_both + 1
p_sbp = c_sbp / denom
p_ldlc = c_ldlc / denom
p_both = c_both / denom
p_non = 1 / denom

In [12]:
# Check that probabilies sum to 1
assert sum(p_sbp + p_ldlc + p_both + p_non) - len(pop0.index) == 0

In [13]:
# Calculate actual initialized fractions
pop0.sbp_medication.notna().sum() / len(pop0)

0.3718

In [14]:
# What was the expected?
p_sbp.mean() + p_both.mean()

0.36765907917039886

In [15]:
pop0.ldlc_medication.notna().sum() / len(pop0)

0.3066

In [16]:
p_ldlc.mean() + p_both.mean()

0.30407625669968147

## medication baseline coverage types


In [18]:
# of those with sbp meds, expect {1: 0.57, 3: 0.43}
pop0.groupby('sbp_medication').size() / pop0.sbp_medication.notna().sum()

sbp_medication
1.0    0.816837
3.0    0.183163
dtype: float64

In [29]:
# of those with ldlc meds, 
# NOTE: level-percent mapping = {1: 0.0382, 2: 0.7194. 4: 0.2424}
pop0.groupby('ldlc_medication').size() / pop0.ldlc_medication.notna().sum()

ldlc_medication
1.0    0.022831
2.0    0.922701
4.0    0.054468
dtype: float64

In [54]:
# Q: these don't seem to match the expected spreads

## scheduled_date TODO

In [36]:
pop0[pop0.scheduled_date.notna()].groupby('visit_type').size()

visit_type
none              504
none;emergency      3
dtype: int64

In [40]:
pop0[pop0.visit_type.str.contains('emergency')]

,tracked,age,exit_time,alive,location,sex,entrance_time,years_of_life_lost,cause_of_death,previous_ischemic_stroke,...,acute_myocardial_infarction_event_count,post_myocardial_infarction_event_time,post_myocardial_infarction_event_count,sbp_medication,sbp_medication_adherence,miss_scheduled_visit_probability,ldlc_medication,ldlc_medication_adherence,visit_type,scheduled_date
2289,True,76.275767,NaT,alive,Alabama,Male,2020-12-04,0.0,not_dead,,...,0,NaT,0,1.0,adherent,0.105546,NaN,adherent,none;emergency,2021-04-17 15:55:47.350444800
3314,True,79.225903,NaT,alive,Alabama,Male,2020-12-04,0.0,not_dead,,...,0,NaT,0,1.0,adherent,0.064111,NaN,adherent,none;emergency,2021-04-05 05:35:49.390051200
4636,True,64.887532,NaT,alive,Alabama,Male,2020-12-04,0.0,not_dead,,...,0,NaT,0,3.0,adherent,0.245843,2.0,primary_non_adherent,none;emergency,2021-05-29 18:04:10.210339200
9820,True,9.978786,NaT,alive,Alabama,Female,2020-12-04,0.0,not_dead,,...,0,NaT,0,NaN,secondary_non_adherent,0.319767,NaN,primary_non_adherent,none;emergency,NaT


In [41]:
pop0[(pop0.ischemic_stroke.str.contains('acute')) | (pop0.myocardial_infarction.str.contains('acute'))][['ischemic_stroke', 'myocardial_infarction', 'visit_type', 'scheduled_date']]

,ischemic_stroke,myocardial_infarction,visit_type,scheduled_date
2289,susceptible_to_ischemic_stroke,acute_myocardial_infarction,none;emergency,2021-04-17 15:55:47.350444800
3314,susceptible_to_ischemic_stroke,acute_myocardial_infarction,none;emergency,2021-04-05 05:35:49.390051200
4636,susceptible_to_ischemic_stroke,acute_myocardial_infarction,none;emergency,2021-05-29 18:04:10.210339200
4990,chronic_ischemic_stroke,acute_myocardial_infarction,none,2021-01-18 06:12:26.655148800
7220,chronic_ischemic_stroke,acute_myocardial_infarction,none,2021-01-14 09:47:52.731916800
9820,acute_ischemic_stroke,susceptible_to_myocardial_infarction,none;emergency,NaT


## medication adherences TODO

# Take one time step
I will expect nothing to have been observed yet

In [22]:
num_steps = 1
i = 1
while i < (num_steps + 1):
    sim.step()
    pop.append(sim.get_population())
    metrics.append(sim.get_value('metrics')(pop[i].index))
    utilization.append(sim.get_value('utilization_rate')(pop[i].index))
    i += 1

2022-09-16 09:49:42.981 | DEBUG    | vivarium.framework.engine:step:172 - 2021-01-01 00:00:00


In [29]:
# check values against notes taken
pop1 = pop[1]

In [40]:
pop1[pop1.alive == 'alive'] # should be 49957

,tracked,exit_time,alive,sex,entrance_time,location,age,cause_of_death,years_of_life_lost,years_lived_with_disability,...,chronic_ischemic_stroke_event_count,susceptible_to_myocardial_infarction_event_time,susceptible_to_myocardial_infarction_event_count,acute_myocardial_infarction_event_count,acute_myocardial_infarction_event_time,post_myocardial_infarction_event_count,post_myocardial_infarction_event_time,miss_scheduled_visit_probability,scheduled_date,visit_type
0,True,NaT,alive,Female,2020-12-04,Alabama,47.465542,not_dead,0.0,0.0,...,0,NaT,0,0,NaT,0,NaT,0.161931,NaT,none
1,True,NaT,alive,Male,2020-12-04,Alabama,41.640854,not_dead,0.0,0.0,...,0,NaT,0,0,NaT,0,NaT,0.262446,NaT,none
2,True,NaT,alive,Female,2020-12-04,Alabama,53.232486,not_dead,0.0,0.0,...,0,NaT,0,0,NaT,0,NaT,0.278118,NaT,none
3,True,NaT,alive,Male,2020-12-04,Alabama,30.534352,not_dead,0.0,0.0,...,0,NaT,0,0,NaT,0,NaT,0.263973,NaT,none
4,True,NaT,alive,Female,2020-12-04,Alabama,19.631466,not_dead,0.0,0.0,...,0,NaT,0,0,NaT,0,NaT,0.120069,NaT,none
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,True,NaT,alive,Male,2020-12-04,Alabama,77.713007,not_dead,0.0,0.0,...,0,NaT,0,0,NaT,0,2020-12-04,0.329822,2021-03-25 22:43:04.260057600,none;background
49996,True,NaT,alive,Male,2020-12-04,Alabama,26.465916,not_dead,0.0,0.0,...,0,NaT,0,0,NaT,0,NaT,0.286116,NaT,none
49997,True,NaT,alive,Female,2020-12-04,Alabama,69.109235,not_dead,0.0,0.0,...,0,NaT,0,0,NaT,0,NaT,0.144109,2021-05-19 15:47:15.924652800,none;background
49998,True,NaT,alive,Female,2020-12-04,Alabama,20.260682,not_dead,0.0,0.0,...,0,NaT,0,0,NaT,0,NaT,0.227279,2021-05-14 22:39:34.116336000,none;background


In [47]:
assert pop1[pop1.alive == 'alive'].visit_type.str.contains('emergency').sum() == 9
assert pop1[pop1.alive == 'alive'].visit_type.str.contains('scheduled').sum() == 724
assert pop1[pop1.alive == 'alive'].visit_type.str.contains('missed').sum() == 73
assert pop1[pop1.alive == 'alive'].visit_type.str.contains('background').sum() == 10849
assert pop1[pop1.alive == 'alive'].scheduled_date.notna().sum() == 12743
assert pop1[pop1.alive == 'alive'].scheduled_date.min() > sim.current_time

In [60]:
# check that metrics have not updates (b/c this first time step is before observatio_start
{k for k in metrics[1] if "healthcare" in k}

# Take a second time step

In [64]:
num_steps = 1
i = 1
while i < (num_steps + 1):
    sim.step()
    pop.append(sim.get_population())
    metrics.append(sim.get_value('metrics')(pop[i].index))
    utilization.append(sim.get_value('utilization_rate')(pop[i].index))
    i += 1

2022-09-16 09:59:54.224 | DEBUG    | vivarium.framework.engine:step:172 - 2021-01-29 00:00:00


In [84]:
# check values against notes taken
pop2 = pop[2]
assert sum(pop2.alive == 'alive') == 49915
assert pop2[pop2.alive == 'alive'].visit_type.str.contains('emergency').sum() == 22
assert pop2[pop2.alive == 'alive'].visit_type.str.contains('scheduled').sum() == 745 - 135
assert pop2[pop2.alive == 'alive'].visit_type.str.contains('missed').sum() == 135
assert pop2[pop2.alive == 'alive'].visit_type.str.contains('background').sum() == 11249
assert pop2[pop2.alive == 'alive'].scheduled_date.notna().sum() == 20701
assert pop2[pop2.alive == 'alive'].scheduled_date.min() > sim.current_time

In [76]:
# check that metrics have updated witih hosptial count and it should match the above
# Note that it should NOT include the first time step b/c that was before observation start!
healthcare_metrics = {k:v for k,v in metrics[2].items() if k in [k for k in metrics[2] if "healthcare" in k]}

In [85]:
# Ensure the metrics match the above checks
assert sum([v for k,v in healthcare_metrics.items() if "emergency" in k]) == 22
assert sum([v for k,v in healthcare_metrics.items() if "scheduled" in k]) == 745 - 135
assert sum([v for k,v in healthcare_metrics.items() if "missed" in k]) == 135
assert sum([v for k,v in healthcare_metrics.items() if "background" in k]) == 11249

# Questions
1. Initializing baseline med spread seems off (sbp and ldlc don't follow table 4.5 Initializatin Parameters
2. Related, after initialization, we have slightly moved away from the baseline coverage parameters (eg, we have simulants in sbp ramp levels 1-4, NOT just levels 1 and 3). Is this the correct approach?
    * Reason is that after initializing the baseline medicine coverage, we then send people to the doctor (post states to schedule 0-6 months out as well as emergency states to schedule 3-6 months out). This means some people may bump from 1->2 and 3->4.
    * Note levels 2 and 4 are indeed a small fraction! That's good - but 1 and 3 are still way off from 57 and 43%, respecitively
    * NOTE: We don't see this affect for ldl-c b/c we haven't implemented those treatment updates yet so it's just what is initialized